In [ ]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    # ———————————————————————————————— camada que recebe todos os dados ————————————————————————————————
    layers.Input(shape=(28,28,3)), # espera uma imagem 28x28 rgb (se for grayscale, 1 ao invés de 3)
    # ——————————————————————————————————————————————————————————————————————————————————————————————————

    # ——————————————————————————————— camada de tratamento inicial ———————————————————————————————
    layers.Resizing(28,28), # reforça o tamanho de 28x28
    layers.Rescaling(1./255), # normaliza os valores de pixel de [0, 255] para [0,1]
    layers.RandomRotation((-0.2, 0.2)), # rotaciona a imagem aleatoriamente, a 0.2 ou -0.2 graus
    # ————————————————————————————————————————————————————————————————————————————————————————————

    # —————————————————————————————————————— camada que limpa a imagem ——————————————————————————————————————
    layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same'),           
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),
    
    # convolution2d/conv2d (quantidade de filtros (quantas vezes a convolução acontece),
    # tamanho do kernel(filtro), strides (deslocamento) e tipo de padding para as bordas (valid/same))
    # convolução = aplicar filtros, dando pesos aos pixels para modificar uma imagem  
    # 
    # maxpooling2d() (dimensão da pool, strides (deslocamento) e tipo de padding para as bordas (valid/same)))
    # max pooling = delimita uma área e pega o maior valor dessa área, reduzindo dimensões, mantendo as
    # informações importantes ao pegar o maior valor e passando esse valor para a próxima camada
    # ———————————————————————————————————————————————————————————————————————————————————————————————————————

    # ————— camada que achata a imagem —————
    layers.Flatten(), # 784
    # ——————————————————————————————————————

    # —————————————————————————————————— camada de treinamento das camadas ——————————————————————————————————
    layers.Dense(64, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),

    layers.Dense(64, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),

    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) # 24

    # dense (units (quantidade de neurônios na camada), activation/função de ativação usada, use_bias=True
    # (por natureza, o uso de bias/viés é ativado), tipo do inicializador dos pesos/kernel (opcional))
    # viés/bias é um parâmetro constante adicional que ajuda o modelo a se ajustar melhor aos dados
    #
    # dense = camada de neurônios completamente conectados aos neurônios da camada anterior
    #
    # ativações = [sigmoid, softmax, relu, etc]
    #
    # dropout(0.2) = aleatoriamente desativa 20% (0.2) dos neurônios da camada anterior
    # durante cada etapa do treinamento para previnir overfitting
    # ———————————————————————————————————————————————————————————————————————————————————————————————————————
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001 #definindo a learning rate (taxa de aprendizado)

# ——————————————————————————————————————— otimizando o treinamento ———————————————————————————————————————
model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

# optimizer = quando o modelo realiza uma previsão calculamos o erro,
# o optimizer é como vamos ajustar os pesos com base no erro calculado
#
# adam = mistura de momentum + adaptativo. é o mais usado, aprende mais rápido e é estável
#
# momentum: reduz as oscilações do resultado da melhora do erro, guarda o histórico dos
# valores resultantes e utiliza a "velocidade" e a direção que está indo para calcular o novo
# adaptativo: utiliza sua própria taxa de aprendizado, se um peso erra muito, o otimizador
# reduz a taxa pra não oscilar tanto, se erra pouco o otimizador aumenta a taxa pra aprender mais rápido
#
# loss = define como vai ser calculado o erro com a predição
# { 'Classificação binária' : 'binary_crossentropy',
#   'Classificação multi-classe (one-hot)' : 'categorical_crossentropy',
#   'Classificação multi-classe (inteiro)' : 'sparse_categorical_crossentropy',
#   'Regressão' : 'mse'`, `'mean_squared_error' }
#
# metrics = não altera o funcionamento em si mas podemos definir o que
# queremos acompanhar enquanto o modelo estiver treinando
# ['accuracy', 'binary_accuracy', 'categorical_accuracy', 'sparse_categorical_accuracy', 
# 'Precision', 'Recall', 'AUC', 'top_k_categorical_accuracy']
# ————————————————————————————————————————————————————————————————————————————————————————————————————————

from tensorflow.keras import utils

path = './Data' # caminho onde estão as imagens de teste e treino
batch_size = 64 # número de imagens/samples que serão processadas

# ——————————— configuração das imagens para treinamento ———————————
train = utils.image_dataset_from_directory(
    directory=path + '/Train',
    shuffle = True,
    seed = 1,
    subset = 'training',
    validation = 0.1,
    image_size = (28,28),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Test',
    shuffle = True,
    seed = 1,
    subset = 'validation',
    validation = 0.1,
    image_size = (28,28),
    batch_size = batch_size
)

# directory = caminho relativo de cada conjunto de imagens
# (treino e teste)
# shuffle = aleatoriza/embaralha as imagens
# seed = semente da aleatorização, garante que se o código
# rodar novamente, será embaralhado da mesma forma
# subset = define os dados, se são de training ou validation
# validation = faz com que 10% das imagens dentro da própria pasta
# especificada sejam separadas para o subset indicado
# ex: se a pasta train tem 100 imagens, 90 serao para training
# e 10 serão para validation
# image_size = tamanho da imagem recebida, aqui é 28x28
# batch_size = a quantidade de amostras que a rede processa de
# uma vez antes de atualizar os pesos durante o treinamento,
# número de imagens que vão em cada batch
# —————————————————————————————————————————————————————————————————

from tensorflow.keras import callbacks

model_path = "./model.keras" # onde o modelo que será treinado nesse código será armazenado

patience = 3 # paciência = quantas vezes o código tolera rodar sem ter melhoras
epochs = 100 # número de épocas treinadas, quantas vezes será o modelo será treinado

# ————————————————————— treinamento do modelo —————————————————————
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

# train = dados de treinamento,
# validation_data = dados de validação/teste
# epochs = quantidade de epochs/épocas, quantas vezes será rodado
# verbose = se true/1, irá mostrar a época e os atributos
# (loss, val_loss, sparse_categorical_accuracy, etc) em texto
#
# callback EarlyStopping = irá parar de rodar as épocas baseado
# na paciência, monitorando val_loss, quando ocorrer o
# early stopping, será mostrado em texto
#
# callback ModelCheckpoint = vai salvar o melhor modelo rodado
# no model.keras (definido no filepath) de acordo com a métrica
# monitorada (nesse caso, loss), salvando tudo, não apenas os
# pesos, onde o loss for o menor possível (mode = 'min')
# —————————————————————————————————————————————————————————————————

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.
Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 2.9302 - sparse_categorical_accuracy: 0.1225 - val_loss: 2.3242 - val_sparse_categorical_accuracy: 0.2471
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.9483 - sparse_categorical_accuracy: 0.3328 - val_loss: 1.7035 - val_sparse_categorical_accuracy: 0.4024
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.4646 - sparse_categorical_accuracy: 0.4893 - val_loss: 1.3525 - val_sparse_categorical_accuracy: 0.5318
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 1.0959 - sparse_categorical_accuracy: 0.6201 - val_loss: 1.0604 - val_sparse_categorical_accuracy: 0.6104
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 0.8073 - sparse_categorical_accuracy: 0.7222 - val_loss: 0.7592 - val_sparse_categorical_accuracy: 0.7345
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - loss: 0.5885 